# Lab 1: Set Covering

First lab + peer review. List this activity in your final report, it will be part of your exam.

## Task

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$,
determine is possible $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$
such that each number between $0$ and $N-1$ appears in at least one list

$$\forall n \in [0, N-1] \ \exists i : n \in L_{s_i}$$

and that the total numbers of elements in all $L_{s_i}$ is minimum.

## Input generator

In [11]:
SEED = 42

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

# Gready Solution (Professor's solution)

In [21]:
import random
import logging
import numpy as np

def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)
    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")


logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000]:
     greedy(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%)
INFO:root:Greedy solution for N=100: w=332 (bloat=232%)
INFO:root:Greedy solution for N=500: w=2162 (bloat=332%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


# Breadth First

In [31]:
import random

def set_covering_problem_bf(universe,subsets):
    elements=set(e for s in subsets for e in s)
    if elements!=universe:
        print("The subsets don't contain the universe.")
        return None
    covered=set()
    solution=[]
    queue=[]
    visited=[]
    while covered!=elements:
        subset = max(subsets,key=lambda s: len(s-covered))
        queue.append(subset)
        x=queue.pop()
        if x not in visited:
            visited.append(subset)
            solution.append(subset)
            covered |= subset
    #print("NUMBER OF VISITED NODES: ",len(visited))
    print("w: ",sum(len(_) for _ in solution))
    return solution

def run_BF(N):
    sub=problem(N,seed=42)
    universe=set(range(N))
    #create a unique set - source: documentation
    subsets=[set(element) for element in sub]
    set_cover=set_covering_problem_bf(universe,subsets)
    #print("SOLUTION: ",set_cover)

for N in [5, 10, 20, 100, 500, 1000]:
     run_BF(N)
     print(N)
     print("___________________________________________")

w:  5
5
___________________________________________
w:  11
10
___________________________________________
w:  29
20
___________________________________________
w:  192
100
___________________________________________
w:  1313
500
___________________________________________
w:  3092
1000
___________________________________________


## Search function
# To-be implemented in the following days...

In [27]:
from typing import Callable
from gx_utils import PriorityQueue


class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

# def result(state, elem):
#     return State(np.array(new_state))

def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0



    while state is not None and not goal_test(state):
        for a in possible_actions(state):
            new_state = result(state, a)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None
    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    #logging.info(f"w= "{sum(...)})
    return list(reversed(path))

## Breadth First with search function
## (To be implemented...)

In [ ]:
parent_state = dict()
state_cost = dict()

logging.getLogger().setLevel(logging.INFO)

#for n in [5,10,20,50,100,500,1000]:
for N in [5]:
    GOAL = set(range(N))
    COVERED = set()

    def goal_test(state):
        return GOAL != COVERED

    input_state = sorted(problem(N, seed=42), key=lambda l: len(l))
    initial_state = State(np.array([]))
    search(
        initial_state=initial_state,
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: len(state_cost),
        unit_cost=lambda a: len(a)
    )